# Big moves on Mondays (Long only)

The strategy suggests that we go long on the S&P 500 on a Monday and close out our position on the Friday that week based on certain conditions. We assume that positions can be taken only in periods when markets are open on Monday and Friday in a week and the Friday in the previous week.

We calculate the following indicators and backtest the conditions shown below:

Calculate the 25-day average of relative_range = (High - Low) / Close and call it rel_range_ma.
The Monday Close must be lower than the previous Friday Close by at least 0.25 times of rel_range_ma.
Create a variable ibs = (Close - Low) / (High - Low). It must be lower than 0.3.
If conditions in 2, and 3 are met, go long on Monday Close.
Square off your position on Friday Close.

Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings 

warnings.filterwarnings('ignore')

Fix the backtesting period

In [2]:
# Backtesting period should be of 5 years
end = pd.datetime.now().date()
start = end - pd.Timedelta(days=1*252)

Create a class and create instance attributes/methods which are required for backtesting

In [3]:
# Create a class
class backtesting_Big_Moves:
    
    # Create Instance attribute and methods
    def __init__(self,ticker,start,end,rel_range_ma,ibs_range):
        
        self.ticker = ticker
        self.start = start
        self.end = end
        self.rel_range_ma = rel_range_ma
        self.ibs_range = ibs_range 
        
        self.fetch_data()
        self.indicators()
        self.signals()
        self.positions()
        self.returns()
        
    #To fetch the required data from yahoo finance    
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end,)
    
    # To compute the required indicators Relative Range MA, IBS, trading day
    def indicators(self):
        self.df['rel_range'] = (self.df['High'] - self.df['Low'])/self.df['Adj Close']
        self.df['rel_range_ma'] = self.df['rel_range'].rolling(window=self.rel_range_ma).mean()
        self.df['day'] = self.df.index.day_name()
        self.df['previous day'] = self.df['day'].shift(1)
        self.df['4 days later'] = self.df['day'].shift(-4)
        self.df['ibs'] = (self.df['Adj Close'] - self.df['Low'])/(self.df['High'] - self.df['Low'])
        
    
        self.df['condition 1'] = np.where((self.df['day'] =='Monday') & (self.df['previous day'] =='Friday') & (self.df['4 days later'] =='Friday'),1,0)
        self.df['condition 2'] = np.where((1 - (self.df['Adj Close']/self.df['Adj Close'].shift())) >= (0.25 * self.df['rel_range_ma']),1,0)
        self.df['condition 3'] = np.where(self.df['ibs'] < self.ibs_range/100,1,0)
        
    # To generate the long only signal
    def signals(self):
        self.df['signals'] = np.where((self.df['condition 1'] == 1) & (self.df['condition 2'] == 1) & (self.df['condition 3'] == 1),1,0)
    
    # To mark the positions taken
    def positions(self):
        self.df['positions'] = self.df['signals'].shift()
        #Positions taken for next three days as well so we have replaced 0 to 1 for the limit of 3
        self.df['positions'] = self.df['positions'].replace(to_replace=0,method='ffill',limit=3)
    
    #To compute the returns generated
    def returns(self):
        self.df['BnH Returns'] = np.log(self.df['Adj Close']/self.df['Adj Close'].shift(1))
        self.df['Strategy Returns'] = self.df['positions'] * self.df['BnH Returns']
        # BnH returns is the returns generated if bought and hold it for the mentioned period simply without any strategy
        # Strategy Returns is the returns generated due to implementation of this strategy on this historical data
        Strat_Returns = self.df['Strategy Returns'].cumsum()[-1]
        BnH_Returns = self.df['BnH Returns'].cumsum()[-1]
        print('Buy and Hold Returns :', self.df['BnH Returns'].cumsum()[-1])
        print('Strategy Returns :', self.df['Strategy Returns'].cumsum()[-1])
        return Strat_Returns, BnH_Returns
    

In [4]:
# Try with SPY index by passing relative range as 25 and internal bar strength as 0.3, this will return the buy and hold return & strategy return 
SPY_backtesting_Big_Moves = backtesting_Big_Moves('SPY',start,end,25,30)

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.04768357676779711
Strategy Returns : 0.06231459914647352


Optimise the strategy by considering few other indices with the various ibs range

In [5]:
ibs_list=[]
Returns=[]
indices=[]
indice_list=['^NSEI','SPY','^CNXIT','^NSEBANK','^GSPC']
for i in range(20,50,10):
    for j in indice_list:
        a = backtesting_Big_Moves(j,start,end,30,i)
        ibs_list.append(i)
        Returns.append(a.returns())
        indices.append(j)
Strat = [x[0] for x in Returns]
BnH = [x[1] for x in Returns]
    

[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.12000278426150326
Strategy Returns : 0.03839739213133887
Buy and Hold Returns : 0.12000278426150326
Strategy Returns : 0.03839739213133887
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.047683650991013515
Strategy Returns : 0.062314599146473755
Buy and Hold Returns : 0.047683650991013515
Strategy Returns : 0.062314599146473755
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.2486620197916904
Strategy Returns : 0.0641865831867213
Buy and Hold Returns : 0.2486620197916904
Strategy Returns : 0.0641865831867213
[*********************100%***********************]  1 of 1 completed
Buy and Hold Returns : 0.09306199825533858
Strategy Returns : 0.019849592352877893
Buy and Hold Returns : 0.09306199825533858
Strategy Returns : 0.019849592352877893
[*********************100%***********************]  1 of 1 complet

In [6]:
# Create a dataframe and validate the returns generated for various indices with different EMA range
Results = pd.DataFrame({'Indices':indices,'IBS':ibs_list,'Strategy Returns':Strat,'BnH Returns':BnH})


In [7]:
# Shows the record in descending order which has maximum return 
Results.sort_values(by='Strategy Returns',inplace=True,ascending=False)

Note : This is simple strategy to learn how to backtest and optimize the parameters by reusing the code.

In [8]:
Results.head(20)

,Indices,IBS,Strategy Returns,BnH Returns
12,^CNXIT,40,0.143353,0.248662
7,^CNXIT,30,0.094151,0.248662
2,^CNXIT,20,0.064187,0.248662
1,SPY,20,0.062315,0.047684
6,SPY,30,0.062315,0.047684
11,SPY,40,0.062315,0.047684
9,^GSPC,30,0.046645,0.039458
0,^NSEI,20,0.038397,0.120003
5,^NSEI,30,0.038397,0.120003
10,^NSEI,40,0.038397,0.120003
